<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/query_engine/knowledge_graph_rag_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Graph RAG Query Engine


## Graph RAG

Graph RAG is an Knowledge-enabled RAG approach to retrieve information from Knowledge Graph on given task. Typically, this is to build context based on entities' SubGraph related to the task.

## GraphStore backed RAG vs VectorStore RAG

As we compared how Graph RAG helps in some use cases in [this tutorial](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html#id1), it's shown Knowledge Graph as the unique format of information could mitigate several issues caused by the nature of the "split and embedding" RAG approach.

## Why Knowledge Graph RAG Query Engine

In Llama Index, there are two scenarios we could apply Graph RAG:

- Build Knowledge Graph from documents with Llama Index, with LLM or even [local models](https://colab.research.google.com/drive/1G6pcR0pXvSkdMQlAK_P-IrYgo-_staxd?usp=sharing), to do this, we should go for `KnowledgeGraphIndex`.
- Leveraging existing Knowledge Graph, in this case, we should use `KnowledgeGraphRAGQueryEngine`.

> Note, the third query engine that's related to KG in Llama Index is `NL2GraphQuery` or `Text2Cypher`, for either exiting KG or not, it could be done with `KnowledgeGraphQueryEngine`.

Before we start the `Knowledge Graph RAG QueryEngine` demo, let's first get ready for basic preparation of Llama Index.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-llms-azure-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-azure-openai
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existi

### OpenAI

In [ ]:
# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = "sk-------"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output


# define LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = None
Settings.chunk_size = 512

Embeddings have been explicitly disabled. Using MockEmbedding.


### Azure

In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# For Azure OpenAI
api_key = "<api-key>"
azure_endpoint = "https://<your-resource-name>.openai.azure.com/"
api_version = "2023-07-01-preview"

llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="my-custom-llm",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="my-custom-embedding",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

#Install nebulagraph-lite
https://github.com/nebula-contrib/nebulagraph-lite

In [ ]:
!pip3 install nebulagraph-lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.5
    Uninstalling psutil-5.9.5:
      Successfully uninstalled psutil-5.9.5


In [ ]:
from nebulagraph_lite import nebulagraph_let as ng_let

n = ng_let()
n.start()

Streaming output truncated to the last 5000 lines.
usr/share/i18n/locales/li_BE
usr/share/i18n/locales/li_NL
usr/share/i18n/locales/lij_IT
usr/share/i18n/locales/lo_LA
usr/share/i18n/locales/lt_LT
usr/share/i18n/locales/lv_LV
usr/share/i18n/locales/mag_IN
usr/share/i18n/locales/mai_IN
usr/share/i18n/locales/mg_MG
usr/share/i18n/locales/mhr_RU
usr/share/i18n/locales/mi_NZ
usr/share/i18n/locales/mk_MK
usr/share/i18n/locales/ml_IN
usr/share/i18n/locales/mn_MN
usr/share/i18n/locales/mni_IN
usr/share/i18n/locales/mr_IN
usr/share/i18n/locales/ms_MY
usr/share/i18n/locales/mt_MT
usr/share/i18n/locales/my_MM
usr/share/i18n/locales/nan_TW@latin
usr/share/i18n/locales/nb_NO
usr/share/i18n/locales/nds_DE
usr/share/i18n/locales/nds_NL
usr/share/i18n/locales/ne_NP
usr/share/i18n/locales/nhn_MX
usr/share/i18n/locales/niu_NU
usr/share/i18n/locales/niu_NZ
usr/share/i18n/locales/nl_AW
usr/share/i18n/locales/nl_BE
usr/share/i18n/locales/nl_BE@euro
usr/share/i18n/locales/nl_NL
usr/share/i18n/locales/nl_NL

In [ ]:
%pip install jupyter_nebulagraph
%load_ext ngql

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.5 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


### Connect and Query NebulaGraph

In [ ]:
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

[OK] Connection Pool Created


,Name
0,basketballplayer


In [ ]:
%ngql USE basketballplayer;
%ngql MATCH (v:player{name:"Tim Duncan"})-->(v2:player) RETURN v2.player.name AS Name;

,Name
0,Tony Parker
1,Manu Ginobili


In [ ]:
%ngql USE basketballplayer;
%ngql MATCH (v:player) RETURN count(v) AS Count;

,Count
0,51


#create llamaindex graph space

In [ ]:
%ngql CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

In [ ]:
%ngql  USE llamaindex;
%ngql  CREATE TAG entity(name string);
%ngql  CREATE EDGE relationship(relationship string);
%ngql CREATE TAG INDEX entity_index ON entity(name(256));

## Create KG from wikidata document for NebulaGraph
https://en.wikipedia.org/wiki/Guardians_of_the_Galaxy_Vol._3

We take [NebulaGraphStore](https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/NebulaGraphKGIndexDemo.html) as an example in this demo, thus before next step to perform Graph RAG on existing KG, let's ensure we have a running NebulaGraph with defined data schema.

This step installs the clients of NebulaGraph, and prepare contexts that defines a [NebulaGraph Graph Space](https://docs.nebula-graph.io/3.6.0/1.introduction/2.data-model/).

In [ ]:
# Create a NebulaGraph (version 3.5.0 or newer) cluster with:
# Option 0 for machines with Docker: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
# Option 1 for Desktop: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext

# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE llamaindex;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# :sleep 10;
# CREATE TAG INDEX entity_index ON entity(name(256));

# %pip install ipython-ngql nebula3-python

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

install wikidata readers (llamaHub intergrators)

In [ ]:
!pip install llama-index-readers-wikipedia
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=aa03351fe90e14927d4eab4c1c6da3c4671c322bd1e22193784fff369e3ae48f
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


###Read Document from Wikidata

In [ ]:
from llama_index.core import download_loader
from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

In [ ]:
# from llama_index.core import download_loader
# from llama_index.readers.wikipedia import WikipediaReader
# loader = WikipediaReader()
# documents = loader.load_data(
#     pages=["The Good Dinosaur"], auto_suggest=False
# )

In [ ]:
len(documents)

1

In [ ]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [ ]:
from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
)

In [ ]:
# install related packages, password is nebula by default
%pip install networkx pyvis

In [ ]:
%ngql USE llamaindex;
%ngql MATCH (s)-[e]->(d) RETURN s,e,d LIMIT 100

,s,e,d
0,"(""Zune mp3 player"" :entity{name: ""Zune mp3 player""})","(""Zune mp3 player"")-[:relationship@2706487797103180146{relationship: ""Created using""}]->(""3d printing"")","(""3d printing"" :entity{name: ""3d printing""})"
1,"(""Yondu"" :entity{name: ""Yondu""})","(""Yondu"")-[:relationship@-8595086201795647612{relationship: ""Resurrecting in""}]->(""Film"")","(""Film"" :entity{name: ""Film""})"
2,"(""Will poulter"" :entity{name: ""Will poulter""})","(""Will poulter"")-[:relationship@-5379878499423044483{relationship: ""Is""}]->(""Adam warlock"")","(""Adam warlock"" :entity{name: ""Adam warlock""})"
3,"(""Will poulter"" :entity{name: ""Will poulter""})","(""Will poulter"")-[:relationship@859166610403217419{relationship: ""Created by""}]->(""Sovereign"")","(""Sovereign"" :entity{name: ""Sovereign""})"
4,"(""Vin diesel"" :entity{name: ""Vin diesel""})","(""Vin diesel"")-[:relationship@-5379878499423044483{relationship: ""Is""}]->(""Member of guardians"")","(""Member of guardians"" :entity{name: ""Member of guardians""})"
5,"(""Vin diesel"" :entity{name: ""Vin diesel""})","(""Vin diesel"")-[:relationship@-5379878499423044483{relationship: ""Is""}]->(""Tree-like humanoid"")","(""Tree-like humanoid"" :entity{name: ""Tree-like humanoid""})"
6,"(""United states"" :entity{name: ""United states""})","(""United states"")-[:relationship@6938521839681444606{relationship: ""Released""}]->(""Film"")","(""Film"" :entity{name: ""Film""})"
7,"(""Thanos"" :entity{name: ""Thanos""})","(""Thanos"")-[:relationship@-9222252242387672454{relationship: ""Instructed""}]->(""Ronan the accuser"")","(""Ronan the accuser"" :entity{name: ""Ronan the accuser""})"
8,"(""Thanos"" :entity{name: ""Thanos""})","(""Thanos"")-[:relationship@5397680562294250685{relationship: ""Trained""}]->(""Nebula"")","(""Nebula"" :entity{name: ""Nebula""})"
9,"(""Tara strong"" :entity{name: ""Tara strong""})","(""Tara strong"")-[:relationship@8612194017249451105{relationship: ""Voices""}]->(""Mainframe"")","(""Mainframe"" :entity{name: ""Mainframe""})"


In [ ]:
# draw the result

%ng_draw

<class 'pyvis.network.Network'> |N|=131 |E|=100

Here, we assumed to have the same Knowledge Graph from [this turtorial](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html#optional-build-the-knowledge-graph-with-llamaindex)

## Perform Graph RAG Query

Finally, let's demo how to do Graph RAG towards an existing Knowledge Graph.

All we need to do is to use `RetrieverQueryEngine` and configure the retriver of it to be `KnowledgeGraphRAGRetriever`.

The `KnowledgeGraphRAGRetriever` performs the following steps:

- Search related Entities of the quesion/task
- Get SubGraph of those Entities (default 2-depth) from the KG
- Build Context based on the SubGraph

Please note, the way to Search related Entities could be either Keyword extraction based or Embedding based, which is controlled by argument `retriever_mode` of the `KnowledgeGraphRAGRetriever`, and supported options are:
- "keyword"
- "embedding"(not yet implemented)
- "keyword_embedding"(not yet implemented)

Here is the example on how to use `RetrieverQueryEngine` and `KnowledgeGraphRAGRetriever`:

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

<ipython-input-36-5e822b6ce702>:4: DeprecationWarning: Call to deprecated class KnowledgeGraphRAGRetriever. (KnowledgeGraphRAGRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  graph_rag_retriever = KnowledgeGraphRAGRetriever(


Then we can query it like:

In [ ]:
from IPython.display import display, Markdown

response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

<b>Peter Quill has a relationship with Arête where he nearly dies trying to cross over from her. He also uses a code and has a spoken interaction with someone named Fuck. Additionally, he has a connection with Kraglin through Arête, where Kraglin fires on Arête after the incident where Quill nearly dies trying to cross over from her. Furthermore, Quill has a delayed leaving interaction with the Guardians involving Arête.</b>

In [ ]:
from IPython.display import display, Markdown

response = query_engine.query(
    "what is the challange of Peter Quill??",
)
display(Markdown(f"<b>{response}</b>"))

<b>Nearly dies trying to cross over from.</b>

In [ ]:
response = await query_engine.aquery(
    "Tell me about the challanges faced Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

<b>Peter Quill faced challenges such as nearly dying while trying to cross over from Arête and dealing with the Guardians delaying their departure. Additionally, he encountered challenges related to the use of Code and interactions with the entity named Fuck.</b>

## Include nl2graphquery as Context in Graph RAG

The nature of (Sub)Graph RAG and nl2graphquery are different. No one is better than the other but just when one fits more in certain type of questions. To understand more on how they differ from the other, see [this demo](https://www.siwei.io/en/demos/graph-rag/) comparing the two.

<video width="938" height="800"
       src="https://github.com/siwei-io/talks/assets/1651790/05d01e53-d819-4f43-9bf1-75549f7f2be9"  
       controls>
</video>

While in real world cases, we may not always know which approach works better, thus, one way to best leverage KG in RAG are fetching both retrieval results as context and letting LLM + Prompt generate answer with them all being involved.

So, optionally, we could choose to synthesise answer from two piece of retrieved context from KG:
- Graph RAG, the default retrieval method, which extracts subgraph that's related to the key entities in the question.
- NL2GraphQuery, generate Knowledge Graph Query based on query and the Schema of the Knowledge Graph, which is by default switched off.

We could set `with_nl2graphquery=True` to enable it like:

In [ ]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

<ipython-input-40-1c29d014ddcc>:1: DeprecationWarning: Call to deprecated class KnowledgeGraphRAGRetriever. (KnowledgeGraphRAGRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(


In [ ]:
response = query_engine_with_nl2graphquery.query(
    "What do you know about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

template
  none is not an allowed value (type=type_error.none.not_allowed)


<b>Peter Quill has a relationship with Arête where he nearly dies trying to cross over from her. He also uses a code and has a spoken interaction with someone named Fuck. Additionally, he has a connection with Kraglin through Arête, where Kraglin fires on Arête after the incident where Quill nearly dies. Furthermore, Quill has a delayed departure with the Guardians after the same incident involving Arête.</b>

And let's check the response's metadata to know more details of the retrival of Graph RAG with nl2graphquery by inspecting `response.metadata`.

- **text2Cypher**, it generates a Cypher Query towards the answer as the context.

```cypher
Graph Store Query: MATCH (e:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e.`entity`.`name` == 'Peter Quill'
RETURN e2.`entity`.`name`
```
- **SubGraph RAG**, it get the SubGraph of 'Peter Quill' to build the context.

- Finally, it combined the two nodes of context, to synthesize the answer.

In [ ]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(response.metadata)

{'27154f0f-1ffa-4a67-b591-385e00b51b19': {'kg_rel_map': {'Quill{name: Quill}': ['Quill{name: '
                                                                                'Quill} '
                                                                                '-[relationship:{relationship: '
                                                                                'Nearly '
                                                                                'dies '
                                                                                'trying '
                                                                                'to '
                                                                                'cross '
                                                                                'over '
                                                                                'from}]-> '
                                                                               